In [1]:
import sys

sys.path.append('./')
from loihi_control_loop import *

## Network construction ##

We construct a 2-layer SNN with 5 neurons in the input layer and 5 neurons in the output layer. To enable online interaction with the spiking neurons on Loihi, we create pseudo connections to the input layer and pseudo spike probe to the output layer. The detailed steps on network construction can be found in `LoihiNetwork.setup_loihi_snn` within `loihi_network.py`.

To find the correct axon id for input spike injection, we need to check the input axon id after compiling the network for Loihi. The function below prints the axon id for 5 neurons in the input layer. Suppose we put the 5 neurons in 5 different cores on Loihi, and the printed results show (x, chip_id, core_id, axon_id). We can use these results to determine where we need to send spikes to in the encoder SNIP.

In [2]:
check_input_layer_axon_id_list(input_core_list=[0, 1, 2, 3, 4])

Input Axon Id List: 
[[(1, 0, 0, 0)], [(1, 0, 1, 0)], [(1, 0, 2, 0)], [(1, 0, 3, 0)], [(1, 0, 4, 0)]]
NoneHost server up... 
192.168.0.101
5777
INFO:DRV:  Running in non-slurm environment on : 192.168.0.101
['ssh', '192.168.0.101', '/home/combra/nxsdk-0.9/nxsdk/bin/arm/nx_driver_server', '192.168.0.100', '5777']
INFO:DRV:  Connecting to 192.168.0.101:33385
INFO:DRV:      Host server up..............Done 0.50s
INFO:DRV:      Encoding axons/synapses.....Done 6.34ms
INFO:DRV:      Compiling Embedded snips....Done 0.55s
INFO:HST:  Args chip=0 cpu=0 /tmp/launcher_chip0_lmt0.bin --chips=1 --remote-relay=1 --epoch=0 
INFO:HST:  Lakemont_driver...
INFO:DRV:      Booting up..................Done 2.57s
INFO:DRV:      Encoding probes.............Done 1.03ms
INFO:HST:  chip=0 cpu=0 halted, status=0x0


We can also put all input neurons on the same core on Loihi. We will use the same input core for our demo.

In [3]:
check_input_layer_axon_id_list(input_core_list=[0, 0, 0, 0, 0])

Input Axon Id List: 
[[(1, 0, 0, 0)], [(1, 0, 0, 1)], [(1, 0, 0, 2)], [(1, 0, 0, 3)], [(1, 0, 0, 4)]]
NoneHost server up... 
192.168.0.101
5777
INFO:DRV:  Running in non-slurm environment on : 192.168.0.101
['ssh', '192.168.0.101', '/home/combra/nxsdk-0.9/nxsdk/bin/arm/nx_driver_server', '192.168.0.100', '5777']
INFO:DRV:  Connecting to 192.168.0.101:46827
INFO:DRV:      Host server up..............Done 0.50s
INFO:DRV:      Compiling Embedded snips....Done 0.50s
INFO:DRV:      Encoding axons/synapses.....Done 6.74ms
INFO:HST:  Args chip=0 cpu=0 /tmp/launcher_chip0_lmt0.bin --chips=1 --remote-relay=1 --epoch=0 
INFO:HST:  Lakemont_driver...
INFO:DRV:      Booting up..................Done 2.57s
INFO:DRV:      Encoding probes.............Done 1.17ms
INFO:HST:  chip=0 cpu=0 halted, status=0x0


## Example 1 ##

**Control-loop**: 1 Control Iteration = 1 Loihi step

**Input**: Host computer gives the list of spikes to encoder SNIP at each Loihi step. Encoder SNIP sends the spikes to corresponding neurons in the input layer.

**Output**: Decoder SNIP record spikes from output layer. Decoder SNIP sends the spikes to host computer at each Loihi step.

In this example, we inject spikes to input neurons at the first 5 steps of Loihi simulation. Since spikes in Loihi require 1 step to reach post-synaptic neurons, the neurons in the output layer will have the same spiking activity 2 steps after the host computer sends the spikes to Loihi.

In [4]:
run_loihi_spike_online_interaction()

NoneHost server up... 
192.168.0.101
5777
INFO:DRV:  Running in non-slurm environment on : 192.168.0.101
['ssh', '192.168.0.101', '/home/combra/nxsdk-0.9/nxsdk/bin/arm/nx_driver_server', '192.168.0.100', '5777']
INFO:DRV:  Connecting to 192.168.0.101:41025
INFO:DRV:      Host server up..............Done 0.51s
INFO:DRV:      Encoding axons/synapses.....Done 1.31ms
INFO:DRV:      Compiling Embedded snips....Done 0.50s
INFO:DRV:      Encoding probes.............Done 1.95ms
INFO:HST:  Args chip=0 cpu=0 /tmp/launcher_chip0_lmt0.bin --chips=1 --remote-relay=1 --epoch=0 
INFO:HST:  Lakemont_driver...
INFO:DRV:      Booting up..................Done 2.57s
INFO:DRV:      Configuring registers.......Done 2.76ms
INFO:DRV:      Transferring probes.........Done 2.63ms
INFO:DRV:      Transferring spikes.........Done 0.39ms
Loihi step:  0  Input Spike:  [1, 0, 0, 0, 0]  Output Spike:  [0, 0, 0, 0, 0]
Loihi step:  1  Input Spike:  [0, 1, 0, 0, 0]  Output Spike:  [0, 0, 0, 0, 0]
Loihi step:  2  Input Sp

## Example 2 ##

**Control-loop**: 1 Control Iteration = 10 Loihi steps

**Input**: Host computer gives the input current to encoder SNIP at each control interation. Encoder SNIP generates spikes using Integrate-and-Fire neurons equations and sends spikes to corresponding neurons in the input layer during the Loihi steps within the control iteration.

**Output**: Decoder SNIP record the number of spikes from output layer during the Loihi steps within the control iteration. Decoder SNIP sends the number of spikes to host computer at each control iteration.

In this example, we inject input currents to Loihi at each control iteration. The number of spikes from the output layers shows the result of 10 Loihi steps. Since spikes in Loihi require 1 step to reach post-synaptic neurons, we need to simulate 12 Loihi steps for each control iteration to get 10 Loihi step results.

In [5]:
run_loihi_window_regular_online_interaction()

NoneHost server up... 
192.168.0.101
5777
INFO:DRV:  Running in non-slurm environment on : 192.168.0.101
['ssh', '192.168.0.101', '/home/combra/nxsdk-0.9/nxsdk/bin/arm/nx_driver_server', '192.168.0.100', '5777']
INFO:DRV:  Connecting to 192.168.0.101:35059
INFO:DRV:      Host server up..............Done 0.44s
INFO:DRV:      Encoding axons/synapses.....Done 1.51ms
INFO:DRV:      Compiling Embedded snips....Done 0.51s
INFO:DRV:      Encoding probes.............Done 0.69ms
INFO:HST:  Args chip=0 cpu=0 /tmp/launcher_chip0_lmt0.bin --chips=1 --remote-relay=1 --epoch=0 
INFO:HST:  Lakemont_driver...
INFO:DRV:      Booting up..................Done 2.57s
INFO:DRV:      Configuring registers.......Done 2.78ms
INFO:DRV:      Transferring probes.........Done 3.20ms
INFO:DRV:      Transferring spikes.........Done 0.35ms
Control step:  0  Input Current:  [10, 0, 0, 0, 0]  Output Spike Number:  [1, 0, 0, 0, 0]
Control step:  1  Input Current:  [0, 20, 0, 0, 0]  Output Spike Number:  [0, 2, 0, 0, 0]


## Example 3 ##

**Control-loop**: 1 Control Iteration = 100 Loihi steps

**Input**: Host computer gives the spike probability to encoder SNIP at each control interation. Encoder SNIP generates Poisson spikes and sends spikes to corresponding neurons in the input layer during the Loihi steps within the control iteration.

**Output**: Same as in example 2.

In this example, we inject spike probability to Loihi at each control iteration. The number of spikes from the output layers shows the result of 100 Loihi steps. Since spikes in Loihi require 1 step to reach post-synaptic neurons, we need to simulate 102 Loihi steps for each control iteration to get 100 Loihi step results.

In [6]:
run_loihi_window_poisson_online_interaction()

NoneHost server up... 
192.168.0.101
5777
INFO:DRV:  Running in non-slurm environment on : 192.168.0.101
['ssh', '192.168.0.101', '/home/combra/nxsdk-0.9/nxsdk/bin/arm/nx_driver_server', '192.168.0.100', '5777']
INFO:DRV:  Connecting to 192.168.0.101:36361
INFO:DRV:      Host server up..............Done 0.50s
INFO:DRV:      Encoding axons/synapses.....Done 1.03ms
INFO:DRV:      Compiling Embedded snips....Done 0.47s
INFO:DRV:      Encoding probes.............Done 2.31ms
INFO:HST:  Args chip=0 cpu=0 /tmp/launcher_chip0_lmt0.bin --chips=1 --remote-relay=1 --epoch=0 
INFO:HST:  Lakemont_driver...
INFO:DRV:      Booting up..................Done 3.17s
INFO:DRV:      Configuring registers.......Done 2.74ms
INFO:DRV:      Transferring probes.........Done 2.80ms
INFO:DRV:      Transferring spikes.........Done 0.52ms
Control step:  0  Input Current:  [10, 0, 0, 0, 0]  Output Spike Number:  [11, 0, 0, 0, 0]
Control step:  1  Input Current:  [0, 20, 0, 0, 0]  Output Spike Number:  [0, 22, 0, 0, 0